In [1]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
from os import getcwd
import numpy as np 
MAX_NB_WORDS=40000 #defines the size of our vocabulary
MAX_SEQUENCE_LENGTH=100
VALIDATION_SPLIT=0.2 #what percent of training examples to be saved for testing
EMBEDDING_DIM=200

## Setting up the Training Data

We need all of our labels to be a simple 1 (offensive) or 0 (not offensive). Given that our dataset has a text label for hate speech and "offensive but not hate" speech, we need to map these text labels to integer ones using the Pandas library. #pandasIsYourDataScienceBestFriend


In [2]:
df = pd.read_csv(getcwd() + '/data/twitter-hate-speech.csv', encoding = "ISO-8859-1")
texts = df[:]['tweet_text']
labels = df[:]['does_this_tweet_contain_hate_speech']
labels = labels.map({'The tweet is not offensive': 0, 
                     'The tweet uses offensive language but not hate speech': 1, 
                     'The tweet contains hate speech': 1})
print("Have", len(labels), "training tweets")

Have 14509 training tweets


## Creating our Tokenizer

A tokenizer will turn each text into a sequence of Integers, with each integer being the index of a token in our dictionary. 

This tokenizer dictionary becomes our vocaulary and we need to cap the length (`MAX_NB_WORDS`) of it as english speakers have an uncanny ability to make up words of low frequency. We ignore these words for reasons of computational limits.

We use `fit_on_text` to make a dictionary from the training texts, while the `texts_to_sequences` transforms each work in the text and replaces it with the corresponding integer value.

We also pad our sequences to make every text sequence the same length. We cap these at 100, which is pretty high given the length of a tweet, but the neural net architecture I "borrowed" below was build for something much larger, so we're doing some padding.

Our labels are going to be a 0 or 1. We only have 1 output, we use `to_categorical` since really we're categorizing as offensive or not offensive.  Thus, the output will be an array (vector) where there is a 1 in the postion for a specific category. We would call that "one-hot" encoding.

Note that we're saving a `word_index`, we'll need that later when we create our embedding layer.

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras import backend as K
K.set_image_dim_ordering('tf')

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Using TensorFlow backend.


/Users/imack/.pyenv/versions/3.5.4/lib/python3.5/site-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 28821 unique tokens.
Shape of data tensor: (14509, 100)
Shape of label tensor: (14509, 2)


## Create the Pickle File

For sanity sakes, we output a text, the sequence which shows the word indexes, and the padded sequence to demostrate. 

From here, we want to dump our dictionary for later use. It becomes part of our model, as we will need it to encode new text we want to perform predictions on. 

In [4]:
import pickle

print(texts[125])
print(sequences[125])
print(data[125])

with open('model/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

@OfficialScarbs @JPizzleFIFA @ItzEmmo omg i said him and charlie look alike
[9586, 9587, 9588, 586, 4, 141, 70, 9, 4384, 105, 495]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0 9586 9587 9588  586    4  141   70    9 4384
  105  495]


## Train/Test Sets

We actually want to split our data into `x_train` (training texts), `y_train` (training labels), `x_test` (test texts), `y_test` (test labels). 

The training examples will be used by our algorithm to train the neural network, but we need to test at every step of the way and make sure we aren't "overfitting".

> *Overfitting "the production of an analysis that corresponds too closely or exactly to a particular set of data, and may therefore fail to fit additional data or predict future observations reliably" -[Oxford Dictionaries](https://www.lexico.com/en/definition/overfitting)*

In [5]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]


## Fits Like a GloVe

Now that we have our vectors, we need to load our pre-trained GloVe word vectors for turning our words into high-demensional vectors for understanding english words. We load the 200-dimensional dataset into a dictionary called `embeddings_index`. Which maps words to arrays of 200 numbers. (this takes a minute to load the data)

[GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)

In [6]:
embeddings_index = {}
f = open('data/glove.twitter.27B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
print(embeddings_index['school'])

Found 1193514 word vectors.
[ 2.1209e-01 -1.2778e-01  4.0272e-01  2.2283e-01 -1.2324e-01  2.0982e-01
  7.9629e-01  5.0014e-01 -7.1023e-01 -3.2143e-01  2.3734e-02 -4.1122e-01
 -3.0436e-01 -2.5120e-01  3.1544e-02 -5.8415e-01 -3.0506e-01 -1.1919e-02
 -8.8954e-01 -1.2058e-01  2.3754e-01  1.9054e-01 -9.7923e-02  2.5295e-01
  3.0038e-01  9.5225e-01 -1.1805e-01 -1.7776e-01 -1.6796e-01 -4.2559e-01
 -1.6858e-01 -6.2006e-01  4.5306e-01 -1.5678e-01 -1.2554e-01  1.2103e-01
  1.9681e-01  1.0356e+00  7.3817e-01 -2.3944e-01 -2.3197e-01  3.3474e-01
 -3.1956e-01 -6.0540e-01  3.9843e-01  2.0453e-01 -2.6106e-01  4.0713e-01
 -1.5686e-01  1.4560e-01  2.5623e-01 -6.0652e-03  5.2414e-02 -4.2158e-01
 -7.4491e-01  8.0630e-02 -7.0024e-01 -6.4837e-01 -3.6998e-01  4.8424e-01
  4.0435e-01  3.5487e-02  8.5513e-02  1.8949e-01 -3.9531e-01  2.9498e-01
  1.1069e-01 -1.4824e-01  6.9241e-02 -3.9568e-01 -5.1378e-01  2.1923e-01
  4.5318e-01  5.9044e-01 -5.5246e-01  2.1597e-01  6.9327e-02 -2.1616e-01
  6.3333e-01  4.1607e-0

## Embedding Layer

The Embedding Layer is the first layer of our "deep learning" network. It take positive integer indexes and turns them into dense vectors based on our GloVe dictionary. Using the `word_index` from our Tokenizer, this is the last step to get our data ready before training. 

In [7]:
from keras.layers import Embedding

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

## Training Our Model

The actual architecture of our model is an Embedding Layer, a Convolutional 1D layer, a polling layer, a n1D Convolutional layer, a pooling layer, a 1D Convlutional layer, a final pooling layer, and a fully connected layer (Dense) connected to the 2 outputs.

This is the real "building" of the model, but honesly I just picked it up from some example on the internet and remove some layers because our tweet window size is so small. We can play with the size of each layer (hyperparameter tuning) to get better results but this can take a lot of experimentation.

We run batches of 128 text examples each iteration for speed, iterating over all examples 5 times (number of epochs). This whole process will set the weights on our neural network.

Each iteraction we can see the accuracy increase until it starts to level off. A common trend with training networks. This model will train really quickly on a CPU, but for large datasets that involved much larger things like images, you'll definitely want to run on a GPU.

In [8]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

print(model.summary())

Training model.
(?, 100, 200)


Train on 11608 samples, validate on 2901 samples
Epoch 1/5


  128/11608 [..............................] - ETA: 1:05 - loss: 0.7339 - acc: 0.5625

  256/11608 [..............................] - ETA: 39s - loss: 0.9949 - acc: 0.4844 

  384/11608 [..............................] - ETA: 30s - loss: 0.9705 - acc: 0.4922

...

11264/11608 [============================>.] - ETA: 0s - loss: 0.1014 - acc: 0.9635

11392/11608 [============================>.] - ETA: 0s - loss: 0.1009 - acc: 0.9637

11520/11608 [============================>.] - ETA: 0s - loss: 0.1007 - acc: 0.9639

11608/11608 [==============================] - 12s 1ms/step - loss: 0.1004 - acc: 0.9639 - val_loss: 0.3332 - val_acc: 0.9004


NameError: name 'mode' is not defined

## Trained Model

We now have a trained model and can run predictions on it! Let's try it out with an example texts.

In [11]:
sequences = tokenizer.texts_to_sequences(['I love talking to you about machine learning'])
padded_input = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

predictions = model.predict(padded_input)
print("Message is", predictions[0][1], "offensive")

Message is 0.046058346 offensive


## Export the Model

Our last step it to explort the model files structure as a json file and its weights as an h5 file. These, along with the pickler dictionary constitute our model.

In [12]:
model_json = model.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model/model.h5")
print("Saved model to disk")


Saved model to disk
